In [1]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
import mxnet as mx
import numpy as np

from sklearn import svm
from sklearn import metrics
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

import os
import sys
import math
import time
import pickle

proj_dir = os.getcwd()
label_dir = os.path.join(proj_dir, 'emotiondetection/features_labels_lld/labels')
lld_dir = os.path.join(proj_dir, 'emotiondetection/features_labels_lld/lld')

pickle_train_y_list = 'pickle_train_y_list'
pickle_train_x_list = 'pickle_train_x_list'

pickle_test_y_list = 'pickle_test_y_list'
pickle_test_x_list = 'pickle_test_x_list'

In [3]:
train_y_list = pickle.load(open(pickle_train_y_list))
train_x_list = pickle.load(open(pickle_train_x_list))

test_y_list = pickle.load(open(pickle_test_y_list))
test_x_list = pickle.load(open(pickle_test_x_list))

In [4]:
def calc_score(test_y_list, predict_y_list):
    print "Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):"
    print 'macro   : ', metrics.precision_recall_fscore_support(test_y_list, predict_y_list, average='macro')
    print 'weighted: ', metrics.precision_recall_fscore_support(test_y_list, predict_y_list, average='weighted')

In [5]:
# z-score normalization
scaler = preprocessing.StandardScaler().fit(train_x_list)
train_x_list = scaler.transform(train_x_list)
test_x_list = scaler.transform(test_x_list)

In [6]:
# PCA whiten
# pca = PCA(n_components='mle', svd_solver='full', whiten=True)
# x_pca = pca.fit(train_x_list)
# train_x_list = pca.transform(train_x_list)
# test_x_list = pca.transform(test_x_list)

In [7]:
# sklearn l1/l2 normalization
# x_normalizer = preprocessing.Normalizer(norm='l2').fit(train_x_list)
# train_x_list = x_normalizer.transform(train_x_list)
# test_x_list = x_normalizer.transform(test_x_list)

In [8]:
# # SMOTE up-sampling training data
# sm = SMOTE(kind='regular')
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)

# # split training data to 5 different classes
# train_x_lists = [[] for i in range(5)]
# for i in xrange(len(train_y_list)):
#     train_x_lists[int(train_y_list[i])].append(train_x_list[i])
# for i in xrange(5):
#     print 'number of class', i, ':', len(train_x_lists[i])

In [9]:
print train_y_list.shape, train_x_list.shape
print test_y_list.shape, test_x_list.shape

(9959,) (9959, 384)
(8257,) (8257, 384)


In [10]:
print train_x_list[0][:10]
print test_x_list[0][:10]

[-1.28200357  0.06173483 -1.28187449 -0.41449729  0.46579774 -1.17798747
  0.06471169 -0.85753636 -0.64435287 -1.27747438]
[-1.13903374 -0.75002097 -1.13420654 -0.43244806 -0.38227578 -1.17669163
  0.04549002 -0.83947808 -0.63455673 -1.16125476]


In [11]:
X = train_x_list
y = train_y_list

In [12]:
batch_size = 200
train_iter = mx.io.NDArrayIter(train_x_list, train_y_list, batch_size=batch_size)
test_iter = mx.io.NDArrayIter(test_x_list, test_y_list, batch_size=batch_size)

In [13]:
fc2_num_hidden_list = [64, 128, 256, 512, 1024]
fc3_num_hidden_list = [5, 10, 20]

for n2 in fc2_num_hidden_list:
    for n3 in fc3_num_hidden_list:
        print '(2-layer-MLP) ', 'fc2_num_hidden:', n2, ';',  'fc3_num_hidden:', n3
        
        data = mx.symbol.Variable('data')
        # data = mx.symbol.Dropout(data=data, p=0.2)
        # fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=2048)
        # act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
        fc2  = mx.symbol.FullyConnected(data = data, name = 'fc2', num_hidden = n2)
        act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")
        fc3  = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=n3)
        mlp  = mx.symbol.SoftmaxOutput(data = fc3, name = 'softmax')
        
        # mx.viz.plot_network(mlp)
        
        print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )
        model = mx.model.FeedForward(
            ctx = mx.cpu(),       # Run on CPU
            symbol = mlp,         # Use the network we just defined
            num_epoch = 200,       # Train for 10 epochs
            learning_rate = 0.001,  # Learning rate
            optimizer = 'adam')
        model.fit(
            X=train_iter,
            eval_data=test_iter)
        print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )
        
        predict_y_score_list = model.predict(test_x_list)
        predict_y_list = np.argmax(predict_y_score_list, axis=1)
        calc_score(test_y_list, predict_y_list)
    
        print '-----------------------'

(2-layer-MLP)  fc2_num_hidden: 64 ; fc3_num_hidden: 5


INFO:root:Start training with [cpu(0)]


start: 2016-12-27 17:02:47


INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=0.285
INFO:root:Epoch[0] Validation-accuracy=0.655833
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=0.134
INFO:root:Epoch[1] Validation-accuracy=0.657262
INFO:root:Epoch[2] Resetting Data Iterator
INFO:root:Epoch[2] Time cost=0.117
INFO:root:Epoch[2] Validation-accuracy=0.659643
INFO:root:Epoch[3] Resetting Data Iterator
INFO:root:Epoch[3] Time cost=0.127
INFO:root:Epoch[3] Validation-accuracy=0.658452
INFO:root:Epoch[4] Resetting Data Iterator
INFO:root:Epoch[4] Time cost=0.104
INFO:root:Epoch[4] Validation-accuracy=0.653214
INFO:root:Epoch[5] Resetting Data Iterator
INFO:root:Epoch[5] Time cost=0.075
INFO:root:Epoch[5] Validation-accuracy=0.651667
INFO:root:Epoch[6] Resetting Data Iterator
INFO:root:Epoch[6] Time cost=0.060
INFO:root:Epoch[6] Validation-accuracy=0.649524
INFO:root:Epoch[7] Resetting Data Iterator
INFO:root:Epoch[7] Time cost=0.057
INFO:root:Epoch[7] Validation-accuracy=0

start: 2016-12-27 17:03:08
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.32072980620991054, 0.3267983957879817, 0.31550263909922505, None)
weighted:  (0.56050736844373594, 0.55673973598159132, 0.55630973649132387, None)
-----------------------
(2-layer-MLP)  fc2_num_hidden: 64 ; fc3_num_hidden: 10
start: 2016-12-27 17:03:08


...

start: 2016-12-27 17:12:03
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.36104238904928654, 0.32301752805538841, 0.33233678975551673, None)
weighted:  (0.57806986757072043, 0.61705219813491585, 0.59084113267624849, None)
-----------------------
(2-layer-MLP)  fc2_num_hidden: 1024 ; fc3_num_hidden: 10
start: 2016-12-27 17:12:03


INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=0.517
INFO:root:Epoch[0] Validation-accuracy=0.631190
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=0.384
INFO:root:Epoch[1] Validation-accuracy=0.634524
INFO:root:Epoch[2] Resetting Data Iterator
INFO:root:Epoch[2] Time cost=0.382
INFO:root:Epoch[2] Validation-accuracy=0.628333
INFO:root:Epoch[3] Resetting Data Iterator
INFO:root:Epoch[3] Time cost=0.520
INFO:root:Epoch[3] Validation-accuracy=0.616071
INFO:root:Epoch[4] Resetting Data Iterator
INFO:root:Epoch[4] Time cost=0.516
INFO:root:Epoch[4] Validation-accuracy=0.605714
INFO:root:Epoch[5] Resetting Data Iterator
INFO:root:Epoch[5] Time cost=0.515
INFO:root:Epoch[5] Validation-accuracy=0.603214
INFO:root:Epoch[6] Resetting Data Iterator
INFO:root:Epoch[6] Time cost=0.516
INFO:root:Epoch[6] Validation-accuracy=0.595595
INFO:root:Epoch[7] Resetting Data Iterator
INFO:root:Epoch[7] Time cost=0.516
INFO:root:Epoch[7] Validation-accuracy=0

start: 2016-12-27 17:14:04
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.35156322723233291, 0.32165525167944387, 0.32853887380161878, None)
weighted:  (0.57183951054377757, 0.61317669855879864, 0.58656039267217641, None)
-----------------------
(2-layer-MLP)  fc2_num_hidden: 1024 ; fc3_num_hidden: 20
start: 2016-12-27 17:14:04


INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=0.486
INFO:root:Epoch[0] Validation-accuracy=0.609524
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=0.417
INFO:root:Epoch[1] Validation-accuracy=0.634167
INFO:root:Epoch[2] Resetting Data Iterator
INFO:root:Epoch[2] Time cost=0.414
INFO:root:Epoch[2] Validation-accuracy=0.633929
INFO:root:Epoch[3] Resetting Data Iterator
INFO:root:Epoch[3] Time cost=0.398
INFO:root:Epoch[3] Validation-accuracy=0.619881
INFO:root:Epoch[4] Resetting Data Iterator
INFO:root:Epoch[4] Time cost=0.396
INFO:root:Epoch[4] Validation-accuracy=0.605119
INFO:root:Epoch[5] Resetting Data Iterator
INFO:root:Epoch[5] Time cost=0.534
INFO:root:Epoch[5] Validation-accuracy=0.597976
INFO:root:Epoch[6] Resetting Data Iterator
INFO:root:Epoch[6] Time cost=0.534
INFO:root:Epoch[6] Validation-accuracy=0.594167
INFO:root:Epoch[7] Resetting Data Iterator
INFO:root:Epoch[7] Time cost=0.533
INFO:root:Epoch[7] Validation-accuracy=0

start: 2016-12-27 17:16:13
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.35797405234214452, 0.32880375113217347, 0.33644324172772272, None)
weighted:  (0.57580328785551227, 0.61172338621775468, 0.58875415475193116, None)
-----------------------


In [19]:
# SMOTE up-sampling training data
sm = SMOTE(kind='regular')
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list, train_y_list)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)

# split training data to 5 different classes
train_x_lists = [[] for i in range(5)]
for i in xrange(len(train_y_list_smote)):
    train_x_lists[int(train_y_list_smote[i])].append(train_x_list_smote[i])
for i in xrange(5):
    print 'number of class', i, ':', len(train_x_lists[i])

INFO:imblearn.base:Compute classes statistics ...
INFO:imblearn.base:5 classes detected: Counter({2: 5590, 1: 2093, 0: 881, 4: 721, 3: 674})
DEBUG:imblearn.base:Finding the 5 nearest neighbours ...
DEBUG:imblearn.base:Create synthetic samples ...
INFO:imblearn.base:Generated 4916 new samples ...
INFO:imblearn.base:Compute classes statistics ...
INFO:imblearn.base:5 classes detected: Counter({2: 5590, 3: 5590, 1: 2093, 0: 881, 4: 721})
DEBUG:imblearn.base:Finding the 5 nearest neighbours ...
DEBUG:imblearn.base:Create synthetic samples ...
INFO:imblearn.base:Generated 4869 new samples ...
INFO:imblearn.base:Compute classes statistics ...
INFO:imblearn.base:5 classes detected: Counter({2: 5590, 3: 5590, 4: 5590, 1: 2093, 0: 881})
DEBUG:imblearn.base:Finding the 5 nearest neighbours ...
DEBUG:imblearn.base:Create synthetic samples ...
INFO:imblearn.base:Generated 4709 new samples ...
INFO:imblearn.base:Compute classes statistics ...
INFO:imblearn.base:5 classes detected: Counter({0: 5590,

number of class 0 : 5590
number of class 1 : 5590
number of class 2 : 5590
number of class 3 : 5590
number of class 4 : 5590


In [24]:
batch_size = 200
train_iter = mx.io.NDArrayIter(train_x_list_smote, train_y_list_smote, batch_size=batch_size)
test_iter = mx.io.NDArrayIter(test_x_list, test_y_list, batch_size=batch_size)

In [25]:
fc2_num_hidden_list = [64, 128, 256, 512, 1024]
fc3_num_hidden_list = [5, 10, 20]

for n2 in fc2_num_hidden_list:
    for n3 in fc3_num_hidden_list:
        print '(2-layer-MLP) ', 'fc2_num_hidden:', n2, ';',  'fc3_num_hidden:', n3
        
        data = mx.symbol.Variable('data')
        # data = mx.symbol.Dropout(data=data, p=0.2)
        # fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=2048)
        # act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
        fc2  = mx.symbol.FullyConnected(data = data, name = 'fc2', num_hidden = n2)
        act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")
        fc3  = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=n3)
        mlp  = mx.symbol.SoftmaxOutput(data = fc3, name = 'softmax')
        
        # mx.viz.plot_network(mlp)
        
        print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )
        model = mx.model.FeedForward(
            ctx = mx.cpu(),       # Run on CPU
            symbol = mlp,         # Use the network we just defined
            num_epoch = 200,       # Train for 10 epochs
            learning_rate = 0.001,  # Learning rate
            optimizer = 'adam')
        model.fit(
            X=train_iter,
            eval_data=test_iter)
        print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )
        
        predict_y_score_list = model.predict(test_x_list)
        predict_y_list = np.argmax(predict_y_score_list, axis=1)
        calc_score(test_y_list, predict_y_list)
    
        print '-----------------------'

INFO:root:Start training with [cpu(0)]
INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=0.179
INFO:root:Epoch[0] Validation-accuracy=0.180000


(2-layer-MLP)  fc2_num_hidden: 64 ; fc3_num_hidden: 5
start: 2016-12-27 18:51:13


INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=0.180
INFO:root:Epoch[1] Validation-accuracy=0.173810
INFO:root:Epoch[2] Resetting Data Iterator
INFO:root:Epoch[2] Time cost=0.176
INFO:root:Epoch[2] Validation-accuracy=0.179048
INFO:root:Epoch[3] Resetting Data Iterator
INFO:root:Epoch[3] Time cost=0.178
INFO:root:Epoch[3] Validation-accuracy=0.195238
INFO:root:Epoch[4] Resetting Data Iterator
INFO:root:Epoch[4] Time cost=0.177
INFO:root:Epoch[4] Validation-accuracy=0.199286
INFO:root:Epoch[5] Resetting Data Iterator
INFO:root:Epoch[5] Time cost=0.176
INFO:root:Epoch[5] Validation-accuracy=0.200714
INFO:root:Epoch[6] Resetting Data Iterator
INFO:root:Epoch[6] Time cost=0.177
INFO:root:Epoch[6] Validation-accuracy=0.202619
INFO:root:Epoch[7] Resetting Data Iterator
INFO:root:Epoch[7] Time cost=0.177
INFO:root:Epoch[7] Validation-accuracy=0.207143
INFO:root:Epoch[8] Resetting Data Iterator
INFO:root:Epoch[8] Time cost=0.179
INFO:root:Epoch[8] Validation-accuracy=0

start: 2016-12-27 18:51:45
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.30233371621572291, 0.33710392070903983, 0.30580646598755978, None)
weighted:  (0.55227535440165365, 0.50030277340438412, 0.52237784818803523, None)
-----------------------
(2-layer-MLP)  fc2_num_hidden: 64 ; fc3_num_hidden: 10
start: 2016-12-27 18:51:45


INFO:root:Epoch[0] Validation-accuracy=0.149167
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=0.176
INFO:root:Epoch[1] Validation-accuracy=0.161548
INFO:root:Epoch[2] Resetting Data Iterator
INFO:root:Epoch[2] Time cost=0.175
INFO:root:Epoch[2] Validation-accuracy=0.180714
INFO:root:Epoch[3] Resetting Data Iterator
INFO:root:Epoch[3] Time cost=0.177
INFO:root:Epoch[3] Validation-accuracy=0.186310
INFO:root:Epoch[4] Resetting Data Iterator
INFO:root:Epoch[4] Time cost=0.176
INFO:root:Epoch[4] Validation-accuracy=0.183333
INFO:root:Epoch[5] Resetting Data Iterator
INFO:root:Epoch[5] Time cost=0.176
INFO:root:Epoch[5] Validation-accuracy=0.195714
INFO:root:Epoch[6] Resetting Data Iterator
INFO:root:Epoch[6] Time cost=0.175
INFO:root:Epoch[6] Validation-accuracy=0.195238
INFO:root:Epoch[7] Resetting Data Iterator
INFO:root:Epoch[7] Time cost=0.176
INFO:root:Epoch[7] Validation-accuracy=0.198095
INFO:root:Epoch[8] Resetting Data Iterator
INFO:root:Epoch[8] Time cos

...

start: 2016-12-27 19:11:18
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.33697437057817298, 0.32392334862095151, 0.32310613830783386, None)
weighted:  (0.56787565017579344, 0.59222477897541481, 0.57532156823497804, None)
-----------------------
(2-layer-MLP)  fc2_num_hidden: 1024 ; fc3_num_hidden: 10
start: 2016-12-27 19:11:18


INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=1.455
INFO:root:Epoch[0] Validation-accuracy=0.197976
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=1.452
INFO:root:Epoch[1] Validation-accuracy=0.192024
INFO:root:Epoch[2] Resetting Data Iterator
INFO:root:Epoch[2] Time cost=1.452
INFO:root:Epoch[2] Validation-accuracy=0.195595
INFO:root:Epoch[3] Resetting Data Iterator
INFO:root:Epoch[3] Time cost=1.477
INFO:root:Epoch[3] Validation-accuracy=0.196071
INFO:root:Epoch[4] Resetting Data Iterator
INFO:root:Epoch[4] Time cost=1.452
INFO:root:Epoch[4] Validation-accuracy=0.195952
INFO:root:Epoch[5] Resetting Data Iterator
INFO:root:Epoch[5] Time cost=1.453
INFO:root:Epoch[5] Validation-accuracy=0.198095
INFO:root:Epoch[6] Resetting Data Iterator
INFO:root:Epoch[6] Time cost=1.454
INFO:root:Epoch[6] Validation-accuracy=0.196548
INFO:root:Epoch[7] Resetting Data Iterator
INFO:root:Epoch[7] Time cost=1.455
INFO:root:Epoch[7] Validation-accuracy=0

start: 2016-12-27 19:16:38
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.34422003951601521, 0.32705695623872277, 0.32820838799307428, None)
weighted:  (0.57148121511572647, 0.60118687174518592, 0.58129759025297323, None)
-----------------------
(2-layer-MLP)  fc2_num_hidden: 1024 ; fc3_num_hidden: 20
start: 2016-12-27 19:16:38


INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=1.501
INFO:root:Epoch[0] Validation-accuracy=0.197619
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=1.499
INFO:root:Epoch[1] Validation-accuracy=0.192262
INFO:root:Epoch[2] Resetting Data Iterator
INFO:root:Epoch[2] Time cost=1.500
INFO:root:Epoch[2] Validation-accuracy=0.193810
INFO:root:Epoch[3] Resetting Data Iterator
INFO:root:Epoch[3] Time cost=1.502
INFO:root:Epoch[3] Validation-accuracy=0.195000
INFO:root:Epoch[4] Resetting Data Iterator
INFO:root:Epoch[4] Time cost=1.502
INFO:root:Epoch[4] Validation-accuracy=0.194881
INFO:root:Epoch[5] Resetting Data Iterator
INFO:root:Epoch[5] Time cost=1.501
INFO:root:Epoch[5] Validation-accuracy=0.198571
INFO:root:Epoch[6] Resetting Data Iterator
INFO:root:Epoch[6] Time cost=1.500
INFO:root:Epoch[6] Validation-accuracy=0.197024
INFO:root:Epoch[7] Resetting Data Iterator
INFO:root:Epoch[7] Time cost=1.502
INFO:root:Epoch[7] Validation-accuracy=0

start: 2016-12-27 19:22:15
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.33569429479701685, 0.32584972778612054, 0.32478087320683591, None)
weighted:  (0.56714349750782433, 0.59331476323119781, 0.57580872051566023, None)
-----------------------


In [28]:
fc2_num_hidden_list = [64, 128, 256, 512, 1024]
fc3_num_hidden_list = [5, 10, 20]

for n2 in fc2_num_hidden_list:
    for n3 in fc3_num_hidden_list:
        print '(2-layer-MLP) ', 'fc2_num_hidden:', n2, ';',  'fc3_num_hidden:', n3
        
        data = mx.symbol.Variable('data')
        # data = mx.symbol.Dropout(data=data, p=0.2)
        # fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=2048)
        # act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
        fc2  = mx.symbol.FullyConnected(data = data, name = 'fc2', num_hidden = n2)
        act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="sigmoid")
        fc3  = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=n3)
        mlp  = mx.symbol.SoftmaxOutput(data = fc3, name = 'softmax')
        
        # mx.viz.plot_network(mlp)
        
        print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )
        model = mx.model.FeedForward(
            ctx = mx.cpu(),       # Run on CPU
            symbol = mlp,         # Use the network we just defined
            num_epoch = 200,       # Train for 10 epochs
            learning_rate = 0.001,  # Learning rate
            optimizer = 'adam')
        model.fit(
            X=train_iter,
            eval_data=test_iter)
        print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )
        
        predict_y_score_list = model.predict(test_x_list)
        predict_y_list = np.argmax(predict_y_score_list, axis=1)
        calc_score(test_y_list, predict_y_list)
    
        print '-----------------------'

INFO:root:Start training with [cpu(0)]


(2-layer-MLP)  fc2_num_hidden: 64 ; fc3_num_hidden: 5
start: 2016-12-27 22:43:14


INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=0.511
INFO:root:Epoch[0] Validation-accuracy=0.093690
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=0.400
INFO:root:Epoch[1] Validation-accuracy=0.182500
INFO:root:Epoch[2] Resetting Data Iterator
INFO:root:Epoch[2] Time cost=0.416
INFO:root:Epoch[2] Validation-accuracy=0.169881
INFO:root:Epoch[3] Resetting Data Iterator
INFO:root:Epoch[3] Time cost=0.627
INFO:root:Epoch[3] Validation-accuracy=0.169762
INFO:root:Epoch[4] Resetting Data Iterator
INFO:root:Epoch[4] Time cost=0.451
INFO:root:Epoch[4] Validation-accuracy=0.171429
INFO:root:Epoch[5] Resetting Data Iterator
INFO:root:Epoch[5] Time cost=0.405
INFO:root:Epoch[5] Validation-accuracy=0.173214
INFO:root:Epoch[6] Resetting Data Iterator
INFO:root:Epoch[6] Time cost=0.401
INFO:root:Epoch[6] Validation-accuracy=0.177143
INFO:root:Epoch[7] Resetting Data Iterator
INFO:root:Epoch[7] Time cost=0.402
INFO:root:Epoch[7] Validation-accuracy=0

start: 2016-12-27 22:44:59
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.30649133092861669, 0.36500588879809065, 0.30441441372216066, None)
weighted:  (0.57805128167442665, 0.44120140486859633, 0.48753495409263242, None)
-----------------------
(2-layer-MLP)  fc2_num_hidden: 64 ; fc3_num_hidden: 10
start: 2016-12-27 22:44:59


INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=0.403
INFO:root:Epoch[0] Validation-accuracy=0.093929
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=0.449
INFO:root:Epoch[1] Validation-accuracy=0.154524
INFO:root:Epoch[2] Resetting Data Iterator
INFO:root:Epoch[2] Time cost=0.482
INFO:root:Epoch[2] Validation-accuracy=0.162500
INFO:root:Epoch[3] Resetting Data Iterator
INFO:root:Epoch[3] Time cost=0.361
INFO:root:Epoch[3] Validation-accuracy=0.170000
INFO:root:Epoch[4] Resetting Data Iterator
INFO:root:Epoch[4] Time cost=0.197
INFO:root:Epoch[4] Validation-accuracy=0.173452
INFO:root:Epoch[5] Resetting Data Iterator
INFO:root:Epoch[5] Time cost=0.191
INFO:root:Epoch[5] Validation-accuracy=0.175357
INFO:root:Epoch[6] Resetting Data Iterator
INFO:root:Epoch[6] Time cost=0.245
INFO:root:Epoch[6] Validation-accuracy=0.176786
INFO:root:Epoch[7] Resetting Data Iterator
INFO:root:Epoch[7] Time cost=0.229
INFO:root:Epoch[7] Validation-accuracy=0

...

start: 2016-12-27 23:25:27


INFO:root:Start training with [cpu(0)]


Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.3342057942886269, 0.36894111449056444, 0.33967783103806409, None)
weighted:  (0.57950430850553669, 0.54220661257115177, 0.55794492931910156, None)
-----------------------
(2-layer-MLP)  fc2_num_hidden: 1024 ; fc3_num_hidden: 20
start: 2016-12-27 23:25:27


INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=1.995
INFO:root:Epoch[0] Validation-accuracy=0.184881
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=1.980
INFO:root:Epoch[1] Validation-accuracy=0.194524
INFO:root:Epoch[2] Resetting Data Iterator
INFO:root:Epoch[2] Time cost=1.678
INFO:root:Epoch[2] Validation-accuracy=0.199167
INFO:root:Epoch[3] Resetting Data Iterator
INFO:root:Epoch[3] Time cost=1.637
INFO:root:Epoch[3] Validation-accuracy=0.201667
INFO:root:Epoch[4] Resetting Data Iterator
INFO:root:Epoch[4] Time cost=1.638
INFO:root:Epoch[4] Validation-accuracy=0.202143
INFO:root:Epoch[5] Resetting Data Iterator
INFO:root:Epoch[5] Time cost=1.667
INFO:root:Epoch[5] Validation-accuracy=0.202143
INFO:root:Epoch[6] Resetting Data Iterator
INFO:root:Epoch[6] Time cost=1.954
INFO:root:Epoch[6] Validation-accuracy=0.202857
INFO:root:Epoch[7] Resetting Data Iterator
INFO:root:Epoch[7] Time cost=1.956
INFO:root:Epoch[7] Validation-accuracy=0

start: 2016-12-27 23:32:30
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.33170607830836685, 0.36070193210151752, 0.33554993694517932, None)
weighted:  (0.58179162380638993, 0.54474990916797872, 0.56057412231107728, None)
-----------------------


In [ ]:
print predict_y_list[:100]

In [ ]:
print test_y_list[:100]